In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [3]:
data = pd.read_hdf('../feature_exploration/features.h5', key='no_preprocessing')

In [4]:
data.Activity.unique()

array(['impaired-walking', 'sitting', 'standing', 'walking',
       'assisted-stand', 'sit-to-stand', 'ascending-stairs',
       'cycling-100W', 'cycling-50W', 'descending-stairs', 'lying',
       'rope-jumping', 'sweeping', 'treadmill-running', 'vacuuming',
       'washing-dishes', 'elevator-down', 'elevator-up', 'jump', 'run',
       'sit', 'sleeping', 'stand', 'walking-downstairs',
       'walking-forward', 'walking-left', 'walking-right',
       'walking-upstairs'], dtype=object)

In [135]:
activities = {
    'walking-forward': 1,
    'run': 0,
    'impaired-walking': 1,
    'treadmill-running': 0,
    'rope-jumping': 0,
    'walking': 1
}
mask = np.zeros(data.shape[0], dtype='bool')
for act in activities:
    mask |= data.Activity == act

d2 = data.loc[mask, :].drop('Label', axis=1)
d2.reset_index(inplace=True)
d2['Label'] = d2.Activity.map(activities)

In [136]:
d2.columns

Index(['index', 'Subject', 'Activity', 'Mean', 'MeanCrossRate', 'StdDev',
       'Skewness', 'Kurtosis', 'Range', 'IQR', 'RMS', 'Autocorrelation',
       'LinearSlope', 'SignalEntropy', 'SampleEntropy', 'PermutationEntropy',
       'ComplexityInvariantDistance', 'RangeCountPercentage',
       'RatioBeyondRSigma', 'JerkMetric', 'DimensionlessJerk', 'SPARC',
       'DominantFrequency', 'DominantFrequencyValue', 'PowerSpectralSum',
       'SpectralFlatness', 'SpectralEntropy', 'DetailPower',
       'DetailPowerRatio', 'Label'],
      dtype='object')

In [137]:
plt.close('all')

In [138]:
f1 = 'IQR'
f2 = 'DetailPowerRatio'

feats = ['MeanCrossRate', 'IQR', 'RMS', 'DimensionlessJerk', 'DominantFrequency', 'PowerSpectralSum', 'SpectralEntropy', 'DetailPowerRatio']

X_ = d2.loc[:, feats].values
y_ = d2.loc[:, 'Label'].values.ravel()

plt.figure(figsize=(5, 3))
plt.scatter(X_[:, 0], X_[:, 1], s=1, c=y_)
plt.xlabel(f1)
plt.ylabel(f2)
plt.colorbar()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [145]:
X = torch.from_numpy(X_).type(torch.FloatTensor)
y = torch.from_numpy(y_).type(torch.LongTensor)

dset = TensorDataset(X, y)
dl = DataLoader(dset, batch_size=256, shuffle=True)

In [140]:
class Net(nn.Module):
    def __init__(self, nf):
        super().__init__()
        self.fc1 = nn.Linear(nf, 3)
        self.fc2 = nn.Linear(3, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.fc2(x)
        return x
    
    def predict(self, x):
        pred = F.softmax(self.forward(x), dim=1)
        ans = []
        for t in pred:
            if t[0] > t[1]:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)

In [141]:
model = Net(X_.shape[1])

crit = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.01)

In [153]:
max_epoch = 15
losses = []

old_loss = 1e12
delta_loss = 1e3

i = 0
while i < max_epoch and delta_loss > 0.02:
    tloss = 0
    for xb, yb in dl:
        y_pred = model.forward(X)

        loss = crit(y_pred, y)
        tloss += loss.item()

        optim.zero_grad()
        loss.backward()
        optim.step()
    
    delta_loss = np.abs((old_loss - tloss) / old_loss)
    old_loss = tloss
    
    losses.append(tloss)
    if i % 2 == 1:
        print(i, tloss, delta_loss)
    i += 1

1 8.514067537102044e-08 0.018066586182182206


In [154]:
from sklearn.metrics import accuracy_score

print(accuracy_score(model.predict(X), y))

1.0


In [131]:
def predict(x):
    #Convert into numpy element to tensor
    x = torch.from_numpy(x).type(torch.FloatTensor)
    #Predict and return ans
    ans = model.predict(x)
    return ans.numpy()

In [132]:
def plot_decision_boundary(pred_func,X,y):
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx,yy=np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.figure()
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], s=2, c=y, cmap=plt.cm.binary)

In [133]:
plot_decision_boundary(lambda x: predict(x), X.numpy(), y.numpy())

RuntimeError: size mismatch, m1: [49680 x 2], m2: [4 x 3] at /opt/conda/conda-bld/pytorch_1591914858187/work/aten/src/TH/generic/THTensorMath.cpp:41